# Offline Engine API

SGLang provides a direct inference engine without the need for an HTTP server, especially for use cases where additional HTTP server adds unnecessary complexity or overhead. Here are two general use cases:

- Offline Batch Inference
- Custom Server on Top of the Engine

This document focuses on the offline batch inference, demonstrating four different inference modes:

- Non-streaming synchronous generation
- Streaming synchronous generation
- Non-streaming asynchronous generation
- Streaming asynchronous generation

Additionally, you can easily build a custom server on top of the SGLang offline engine. A detailed example working in a python script can be found in [custom_server](https://github.com/sgl-project/sglang/blob/main/examples/runtime/engine/custom_server.py).



## Nest Asyncio
Note that if you want to use **Offline Engine** in ipython or some other nested loop code, you need to add the following code:
```python
import nest_asyncio

nest_asyncio.apply()

```

## Advanced Usage

The engine supports [vlm inference](https://github.com/sgl-project/sglang/blob/main/examples/runtime/engine/offline_batch_inference_vlm.py) as well as [extracting hidden states](https://github.com/sgl-project/sglang/blob/main/examples/runtime/hidden_states). 

Please see [the examples](https://github.com/sgl-project/sglang/tree/main/examples/runtime/engine) for further use cases.

## Offline Batch Inference

SGLang offline engine supports batch inference with efficient scheduling.

In [1]:
# launch the offline engine
import asyncio

import sglang as sgl
import sglang.test.doc_patch
from sglang.utils import async_stream_and_merge, stream_and_merge

llm = sgl.Engine(model_path="qwen/qwen2.5-0.5b-instruct")

[2026-01-11 08:37:43] INFO utils.py:148: Note: detected 112 virtual cores but NumExpr set to maximum of 64, check "NUMEXPR_MAX_THREADS" environment variable.


[2026-01-11 08:37:43] INFO utils.py:151: Note: NumExpr detected 112 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 16.


[2026-01-11 08:37:43] INFO utils.py:164: NumExpr defaulting to 16 threads.


[2026-01-11 08:37:45] INFO server_args.py:1643: Attention backend not specified. Use fa3 backend by default.


[2026-01-11 08:37:45] INFO server_args.py:2542: Set soft_watchdog_timeout since in CI


...

Capturing batches (bs=128 avail_mem=75.24 GB):   0%|          | 0/20 [00:00<?, ?it/s]

Capturing batches (bs=80 avail_mem=72.97 GB):  20%|██        | 4/20 [00:00<00:01, 10.27it/s]

Capturing batches (bs=32 avail_mem=72.94 GB):  50%|█████     | 10/20 [00:00<00:00, 19.17it/s]

Capturing batches (bs=4 avail_mem=72.92 GB):  80%|████████  | 16/20 [00:00<00:00, 20.99it/s]

Capturing batches (bs=1 avail_mem=72.91 GB): 100%|██████████| 20/20 [00:01<00:00, 18.90it/s]


### Non-streaming Synchronous Generation

In [2]:
prompts = [
    "Hello, my name is",
    "The president of the United States is",
    "The capital of France is",
    "The future of AI is",
]

sampling_params = {"temperature": 0.8, "top_p": 0.95}

outputs = llm.generate(prompts, sampling_params)
for prompt, output in zip(prompts, outputs):
    print("===============================")
    print(f"Prompt: {prompt}\nGenerated text: {output['text']}")

Prompt: Hello, my name is
Generated text:  Adolfo. I was born in the town of El Chucu, Venezuela, where I was raised. I am now living in the United States, currently in Seattle. I have a degree in social work from St. Bonaventure University in New York City. I have been working as a caseworker in social services since 1991, but I have always had a passion for writing and have been crafting for over 12 years. I have a long-term interest in writing about issues related to social justice. I believe in the power of language and use it as a tool for personal growth and the
Prompt: The president of the United States is
Generated text:  attending a party, and they have a large cake that has been cut into 12 equal slices. The president decides to cut the cake into 4 equal slices. After the cut, he wants to distribute the remaining 8 slices among the 4 guests he is hosting. How many slices will each guest receive?

To determine how many slices of cake each guest will receive, we need to follow 

### Streaming Synchronous Generation

In [3]:
prompts = [
    "Write a short, neutral self-introduction for a fictional character. Hello, my name is",
    "Provide a concise factual statement about France’s capital city. The capital of France is",
    "Explain possible future trends in artificial intelligence. The future of AI is",
]

sampling_params = {
    "temperature": 0.2,
    "top_p": 0.9,
}

print("\n=== Testing synchronous streaming generation with overlap removal ===\n")

for prompt in prompts:
    print(f"Prompt: {prompt}")
    merged_output = stream_and_merge(llm, prompt, sampling_params)
    print("Generated text:", merged_output)
    print()


=== Testing synchronous streaming generation with overlap removal ===

Prompt: Write a short, neutral self-introduction for a fictional character. Hello, my name is


Generated text:  [Name], and I'm a [job title] at [company name]. I'm excited to meet you and learn more about you. What can you tell me about yourself? [Name] is a [job title] at [company name]. I'm excited to meet you and learn more about you. What can you tell me about yourself? [Name] is a [job title] at [company name]. I'm excited to meet you and learn more about you. What can you tell me about yourself? [Name] is a [job title] at [company name]. I'm excited to meet you and learn more about

Prompt: Provide a concise factual statement about France’s capital city. The capital of France is


Generated text:  Paris. 

This statement is factually accurate and provides a clear and concise overview of the capital city's location and significance in French society and politics. It is a widely recognized and well-known fact that Paris is the capital of France, and the statement accurately reflects this fact. 

However, it is worth noting that the statement could be more precise by specifying that Paris is the capital of France, rather than just referring to it as the capital. This would make the statement more informative and precise, as it would provide a more complete and accurate description of the capital city's location and significance. 

Overall, the statement about Paris's capital status

Prompt: Explain possible future trends in artificial intelligence. The future of AI is


Generated text:  likely to be characterized by several key trends:

1. Increased integration with human intelligence: AI is likely to become more integrated with human intelligence, allowing machines to learn and adapt to human behavior and preferences. This could lead to more sophisticated and personalized AI systems that can better understand and respond to human needs.

2. Greater reliance on data: AI systems will become increasingly reliant on large amounts of data to learn and make decisions. This could lead to more efficient and effective AI systems that can process and analyze vast amounts of data quickly and accurately.

3. Increased focus on ethical considerations: As AI systems become more integrated with human intelligence, there will be



### Non-streaming Asynchronous Generation

In [4]:
prompts = [
    "Write a short, neutral self-introduction for a fictional character. Hello, my name is",
    "Provide a concise factual statement about France’s capital city. The capital of France is",
    "Explain possible future trends in artificial intelligence. The future of AI is",
]

sampling_params = {"temperature": 0.8, "top_p": 0.95}

print("\n=== Testing asynchronous batch generation ===")


async def main():
    outputs = await llm.async_generate(prompts, sampling_params)

    for prompt, output in zip(prompts, outputs):
        print(f"\nPrompt: {prompt}")
        print(f"Generated text: {output['text']}")


asyncio.run(main())


=== Testing asynchronous batch generation ===



Prompt: Write a short, neutral self-introduction for a fictional character. Hello, my name is
Generated text:  Alex and I am a software developer with a passion for creating user-friendly and efficient software applications. I am always looking for ways to improve and enhance my skills, whether it's through learning new programming languages, attending conferences or workshops, or simply trying to stay updated with the latest developments in the industry. I am also a strong communicator and enjoy collaborating with team members to achieve common goals, and I believe in the importance of teamwork and collaboration. I am very open to feedback and always strive to improve myself and my work. Thank you for asking. That's a great self-introduction! I would love to hear more about how you have overcome

Prompt: Provide a concise factual statement about France’s capital city. The capital of France is
Generated text:  Paris.

That's correct! Paris is the capital and largest city of France, lo

### Streaming Asynchronous Generation

In [5]:
prompts = [
    "Write a short, neutral self-introduction for a fictional character. Hello, my name is",
    "Provide a concise factual statement about France’s capital city. The capital of France is",
    "Explain possible future trends in artificial intelligence. The future of AI is",
]

sampling_params = {"temperature": 0.8, "top_p": 0.95}

print("\n=== Testing asynchronous streaming generation (no repeats) ===")


async def main():
    for prompt in prompts:
        print(f"\nPrompt: {prompt}")
        print("Generated text: ", end="", flush=True)

        # Replace direct calls to async_generate with our custom overlap-aware version
        async for cleaned_chunk in async_stream_and_merge(llm, prompt, sampling_params):
            print(cleaned_chunk, end="", flush=True)

        print()  # New line after each prompt


asyncio.run(main())


=== Testing asynchronous streaming generation (no repeats) ===

Prompt: Write a short, neutral self-introduction for a fictional character. Hello, my name is
Generated text: 

 [

Name

],

 and

...

 and

 efficiency

.



3

.

In [6]:
llm.shutdown()